# Notebook for visualizing grids scheme on map

In [25]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder (e.g., 'masteroppgave') by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Find and append 'masteroppgave' to sys.path dynamically
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

Project root found: /Users/thomasnitsche/Developer/skole/master/code/masteroppgave


In [26]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from schemes.lsh_grid import GridLSH
from constants import  *
from utils.helpers import file_handler
import pprint

### Define Grid

In [27]:
Grid = GridLSH(
        "G1",
        min_lat=R_MIN_LAT,
        max_lat=R_MAX_LAT,
        min_lon=R_MIN_LON,
        max_lon=R_MAX_LON,
        resolution=0.7,
        layers=5,
        meta_file="meta.txt",
        data_path="/data",
    )
print(Grid)

Grid: G1
Covering: (5.559754011676299, 7.451072531046803) km 
Resolution: 0.7 km 
Distortion: [0.08586922964381963, 0.549651076505244, 0.37353029466037885, 0.5019433629981879, 0.06889756586350973] km 
Dimensions: (7, 10) cells


### Visualize Grid on map

In [28]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_grid_with_boundary(grid_lsh):
    """
    Visualizes the grid of the GridLSH object using folium and adds a bounding box.
    
    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    
    Returns:
    - A folium map object.
    """
    
    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(grid_lsh.min_lat, grid_lsh.min_lon), (grid_lsh.max_lat, grid_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each layer
        layer_group = FeatureGroupSubGroup(base_layer, name=f"Layer {layer_index + 1}")
        map_grid.add_child(layer_group)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(layer_group)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid

# Example usage
grid_map = visualize_grid_with_boundary(Grid)
grid_map.save("grid_visualization.html")


## Add trajectory

### Single trajectory

In [29]:
trajs_1 = file_handler.load_all_trajectory_files(ROME_OUTPUT_FOLDER_FULL, "R")

trajs_2 = file_handler.load_trajectory_files(["R_ABA.txt"], ROME_OUTPUT_FOLDER_FULL) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values
print(trajs_2)

{'R_ABA': [[41.9179142561302, 12.491784351789], [41.9180006706546, 12.4915482295984], [41.9180164819596, 12.4916092393588], [41.9180202722101, 12.4916823873694], [41.9180643901914, 12.4916340149177], [41.917986210609, 12.4916812600513], [41.9180102666333, 12.4915954431524], [41.9180476928257, 12.4915810014137], [41.9180308950437, 12.4916438614414], [41.918021115009, 12.4915702289865], [41.9180586784783, 12.491518610133], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9180586591962, 12.4915187226679], [41.9

### Hash trajectory

In [30]:
hash_ = Grid._create_trajectory_hash(trajs_2["R_ABA"])

print(hash_)

[[[41.918760189824404, 12.491710053746854], [41.918760189824404, 12.500155723850932], [41.918760189824404, 12.491710053746854], [41.91242958769371, 12.491710053746854], [41.906098985563005, 12.491710053746854], [41.89976838343231, 12.491710053746854]], [[41.91662389961931, 12.488860024326849], [41.91662389961931, 12.49730569443093], [41.922954501750006, 12.488860024326849], [41.91662389961931, 12.488860024326849], [41.922954501750006, 12.488860024326849], [41.922954501750006, 12.49730569443093], [41.91662389961931, 12.49730569443093], [41.91029329748861, 12.49730569443093], [41.91029329748861, 12.488860024326849], [41.90396269535791, 12.488860024326849], [41.90396269535791, 12.49730569443093], [41.90396269535791, 12.488860024326849], [41.89763209322721, 12.488860024326849]], [[41.91503111305244, 12.495180754399595], [41.92136171518314, 12.495180754399595], [41.91503111305244, 12.495180754399595], [41.92136171518314, 12.495180754399595], [41.91503111305244, 12.495180754399595], [41.9087

### Visualize Grid, original trajectory and hashed trajectory on map

In [32]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_grid_with_trajectory(grid_lsh, trajectory, hashed_trajectory):
    """
    Visualizes the grid of the GridLSH object using Folium and adds a single trajectory.

    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    - trajectory (list): List of (lat, lon) coordinates representing a single trajectory.
    - hashed_trajectory (list of lists): Hashed trajectory representation per layer.


    Returns:
    - A Folium map object.
    """

    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different grid layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(grid_lsh.min_lat, grid_lsh.min_lon), (grid_lsh.max_lat, grid_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each layer
        layer_group = FeatureGroupSubGroup(base_layer, name=f"Grid Layer {layer_index + 1}")
        map_grid.add_child(layer_group)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(layer_group)

    # Plot the original trajectory as a blue polyline
    folium.PolyLine(
        trajectory,
        color="blue",
        weight=2.5,
        opacity=1,
        popup="Trajectory Path"
    ).add_to(base_layer)

    # Add markers for each point along the original trajectory
    for lat, lon in trajectory:
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,  # Small marker
            color="black",
            fill=True,
            fill_opacity=1,
            popup=f"Point: ({lat:.5f}, {lon:.5f})"
        ).add_to(base_layer)
        
    # Plot hashed trajectory points as markers with circles
    hashed_layer_colors = ["red", "blue", "green", "purple", "orange"]  # Colors for different layers

    for layer_index, layer_points in enumerate(hashed_trajectory):
        layer_color = hashed_layer_colors[layer_index % len(hashed_layer_colors)]  # Assign color per layer

        for lat, lon in layer_points:
            # Add a small circle around the hashed point
            folium.Circle(
                location=(lat, lon),
                radius=20,  # Small circle
                color=layer_color,
                fill=True,
                fill_opacity=0.4,
                popup=f"Layer {layer_index + 1} Hashed Point"
            ).add_to(base_layer)

            # Add a marker on top of the hashed point
            folium.Marker(
                location=(lat, lon),
                icon=folium.Icon(color=layer_color, icon="info-sign"),
                popup=f"Hashed Point ({lat:.5f}, {lon:.5f}) - Layer {layer_index + 1}"
            ).add_to(base_layer)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid




grid_map = visualize_grid_with_trajectory(Grid, trajs_2["R_ABA"], hash_)
grid_map.save("grid_with_trajectory.html")
